In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from sklearn.metrics import mean_squared_error
%matplotlib inline
np.set_printoptions(suppress=True)   
pd.set_option('display.float_format', lambda x: '%.5f' % x) 
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)
import pandas as pd
import re
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns
%matplotlib inline
np.set_printoptions(suppress=True)   
import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestRegressor
from matplotlib.ticker import FuncFormatter
from matplotlib.pyplot import MultipleLocator
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
import joblib
import scipy.stats
from time import *
from tqdm import tnrange, tqdm_notebook
from time import sleep
from matplotlib.mathtext import _mathtext as mathtext
from sklearn.utils import resample
from sklearn.ensemble import VotingRegressor
mathtext.FontConstantsBase.sup1 = 0.3 

Please forgive my inelegant code

In [2]:
var = locals()

In [3]:
nh3 = pd.read_excel('E:\\OneDrive - HKUST Connect\\Research\\nh3_ml\\Global_EF\\df_training_2627_space10fold.xlsx')

In [4]:
nh3['Number of fertilizer applications'] = [1 if x >= 2 else 0 for x in nh3['Number of fertilizer applications']]

In [5]:
stratify_columns = ['Fertilizer type', 'Nitrogen placement', 'Number of fertilizer applications', 'Soil tillage practices', 'Crop type']
bootstrapped_datasets = []


for i in range(5,104):
    var['nh3_' + str(i)] = resample(nh3, stratify=nh3[stratify_columns], replace=True,random_state = i)
    bootstrapped_datasets.append(var['nh3_' + str(i)])
    #print(var['nh3_' + str(i)]['EF'].mean())
    #print(var['nh3_' + str(i)]['Number of fertilizer applications'].sum())
    #print(len(var['nh3_' + str(i)][var['nh3_' + str(i)]['Crop type'] == 'Rice']))
bootstrapped_datasets.append(nh3)

In [304]:
len(bootstrapped_datasets)

100

In [312]:
nh3_orig = nh3.copy()

In [321]:
for nh3df in bootstrapped_datasets:    
    for i in range(1, len(nh3df['Fertilizer type'].unique()) + 1) :
        nh3df[nh3df['Fertilizer type'].unique()[i - 1]] = np.array(nh3df['Fertilizer type'] == nh3df['Fertilizer type'].unique()[i - 1]).astype(np.int32)
        i += 1
    del nh3df['Fertilizer type']

In [322]:
for nh3df in bootstrapped_datasets:   
    for i in range(1, len(nh3df['Nitrogen placement'].unique()) + 1) :
        nh3df[nh3df['Nitrogen placement'].unique()[i - 1]] = np.array(nh3df['Nitrogen placement'] == nh3df['Nitrogen placement'].unique()[i - 1]).astype(np.int32)
        i += 1
    del nh3df['Nitrogen placement']

In [323]:
for nh3df in bootstrapped_datasets:   
    nh3df['Soil tillage practices'] = [1 if x=='NT' else 0 for x in nh3df['Soil tillage practices']]

In [324]:
for nh3df in bootstrapped_datasets:   
    for i in range(1, len(nh3df['Crop type'].unique()) + 1) :
        nh3df[nh3df['Crop type'].unique()[i - 1]] = np.array(nh3df['Crop type'] == nh3df['Crop type'].unique()[i - 1]).astype(np.int32)
        i += 1
    del nh3df['Crop type']

In [329]:
for nh3df in bootstrapped_datasets[:-1]: 
    nh3df = sklearn.utils.shuffle(nh3df,random_state=2022)  
    nh3df.index = range(0,len(nh3df))
    nh3df

In [6]:
nh3 = sklearn.utils.shuffle(nh3,random_state=2022)  #数据乱序
nh3.index = range(0,len(nh3))

In [333]:
EFraw = nh3['EF'].copy()
#nh3['EF'] = np.log(nh3['EF'])
pt = preprocessing.PowerTransformer(method='box-cox', standardize=False)
nh3['EF'] = pt.fit_transform(nh3['EF'].ravel().reshape(-1, 1))

In [334]:
for nh3df in bootstrapped_datasets[:-1]: 
    nh3df['EF'] = pt.transform(nh3df['EF'].ravel().reshape(-1, 1))

In [339]:
X_col_ori = list(nh3.columns)

In [340]:
X_col_ori.remove('EF')
X_col_ori.remove('region')
X_col_ori.remove('Longitude')
X_col_ori.remove('Latitude')

In [341]:
X = nh3[X_col_ori]

In [342]:
for i in range(5,104):
    var['X_'+str(i)] = var['nh3_' + str(i)][X_col_ori]
    var['X_'+str(i)] = var['X_'+str(i)].rename(columns={'Number of fertilizer applications':'FAT',
                      'Soil tillage practices':'STP',
                      'Average daily air temperature':'Tem',
                      'SOC(g/kg)':'SOC','TN(g/kg)':'TN','BD(g/cm3)':'BD','clay(%)':'Clay','CEC(cmol/kg)':'CEC',
                      'Nitrogen application rate':'Nrate',
                      'U':'UOA','P+I':'Water'})

In [344]:
X = X.rename(columns={'Number of fertilizer applications':'FAT',
                      'Soil tillage practices':'STP',
                      'Average daily air temperature':'Tem',
                      'SOC(g/kg)':'SOC','TN(g/kg)':'TN','BD(g/cm3)':'BD','clay(%)':'Clay','CEC(cmol/kg)':'CEC',
                      'Nitrogen application rate':'Nrate',
                      'U':'UOA','P+I':'Water'})

In [345]:
X_col = list(X.columns)

In [348]:
Y = nh3['EF'].values.reshape(len(nh3),1)

In [349]:
for i in range(5,104):
    var['Y_'+str(i)] = var['nh3_' + str(i)]['EF'].values.reshape(len(var['nh3_' + str(i)]),1)

In [350]:
X.shape, Y.shape

((2627, 22), (2627, 1))

In [351]:
var['X_'+str(i)].shape, var['Y_'+str(i)].shape

((2627, 22), (2627, 1))

In [371]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state = 266) 

In [373]:
x_train.shape, x_test.shape

((2101, 22), (526, 22))

In [374]:
y_train.shape, y_test.shape

((2101, 1), (526, 1))

In [375]:
n_estimators = range(100,101)
random_state = range(2023,2024)
max_features= range(1,22)
min_samples_leaf = range(1,5)
max_depth = [3, 5, 8, 10, 15, 20, 25, 30, 35, 40, None] 

In [376]:
param_grid = {'n_estimators': n_estimators,
              'max_features': max_features,
             'random_state': random_state,
             'min_samples_leaf': min_samples_leaf,
             'max_depth': max_depth}

In [377]:
grid = GridSearchCV(RandomForestRegressor(), param_grid, cv=10,n_jobs = -1,return_train_score=False)

In [378]:
begin_time = time()

In [379]:
grid.fit(x_train, y_train.ravel())
#grid.fit(X, Y.ravel())

GridSearchCV(cv=10, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [3, 5, 8, 10, 15, 20, 25, 30, 35, 40,
                                       None],
                         'max_features': range(1, 22),
                         'min_samples_leaf': range(1, 5),
                         'n_estimators': range(100, 101),
                         'random_state': range(2023, 2024)})

In [380]:
end_time = time()
run_time = end_time-begin_time
print ('time cost：',run_time/60, 'min') 

time cost： 6.464568634827931 min


In [381]:
grid.best_params_

{'max_depth': 25,
 'max_features': 17,
 'min_samples_leaf': 1,
 'n_estimators': 100,
 'random_state': 2023}

In [382]:
n_estimators = range(500,501)
random_state = range(2023,2024)
max_features= range(6,22)
min_samples_leaf = range(1,2)
max_depth = [15, 20, 25, 30, 35, 40, None]

In [383]:
param_grid = {'n_estimators': n_estimators,
              'max_features': max_features,
             'random_state': random_state,
             'min_samples_leaf': min_samples_leaf,
             'max_depth': max_depth}

In [384]:
grid = GridSearchCV(RandomForestRegressor(), param_grid, cv=10,n_jobs = 7,return_train_score=False)

In [385]:
begin_time = time()

In [386]:
grid.fit(x_train, y_train.ravel())
#grid.fit(X, Y.ravel())

GridSearchCV(cv=10, estimator=RandomForestRegressor(), n_jobs=7,
             param_grid={'max_depth': [15, 20, 25, 30, 35, 40, None],
                         'max_features': range(6, 22),
                         'min_samples_leaf': range(1, 2),
                         'n_estimators': range(500, 501),
                         'random_state': range(2023, 2024)})

In [387]:
end_time = time()
run_time = end_time-begin_time
print ('time cost：',run_time/60, 'min') 

time cost： 8.312330182393392 min


In [388]:
grid.best_params_

{'max_depth': 20,
 'max_features': 11,
 'min_samples_leaf': 1,
 'n_estimators': 500,
 'random_state': 2023}

In [389]:
grid.score(x_test, y_test)

0.793153978833463

In [392]:
grid.score(x_train, y_train)

0.9190411677147101

In [393]:
np.sqrt(mean_squared_error(grid.predict(x_test), y_test))

0.8792060120099658

In [394]:
np.sqrt(mean_squared_error(grid.predict(X), Y))

0.6354907500245996

In [396]:
pd.DataFrame(grid.cv_results_).iloc[grid.best_index_,:]

mean_fit_time                                                                                                         3.06346
std_fit_time                                                                                                          0.05099
mean_score_time                                                                                                       0.06938
std_score_time                                                                                                        0.01441
param_max_depth                                                                                                            20
param_max_features                                                                                                         11
param_min_samples_leaf                                                                                                      1
param_n_estimators                                                                                                    

In [397]:
RF = RandomForestRegressor(n_estimators=grid.best_params_['n_estimators'], max_features=grid.best_params_['max_features'],min_samples_leaf = grid.best_params_['min_samples_leaf'],max_depth = grid.best_params_['max_depth'],oob_score = True,random_state=grid.best_params_['random_state'],n_jobs=14)
RF.fit(x_train, y_train.ravel())

RandomForestRegressor(max_depth=20, max_features=11, n_estimators=500,
                      n_jobs=14, oob_score=True, random_state=2023)

In [398]:
RF

RandomForestRegressor(max_depth=20, max_features=11, n_estimators=500,
                      n_jobs=14, oob_score=True, random_state=2023)

In [399]:
joblib.dump(RF,'E:\\OneDrive - HKUST Connect\\Research\\nh3_ml\\model\\RF_V6_alldatatraining_trainingset_100bootstrap_1000Nmax.pkl') #save model

['E:\\OneDrive - HKUST Connect\\Research\\nh3_ml\\model\\RF_V6_alldatatraining_trainingset_100bootstrap_1000Nmax.pkl']

In [400]:
RF.score(x_test, y_test)

0.793153978833463

In [401]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import max_error
from quantile_forest import RandomForestQuantileRegressor

In [402]:
sdf = []  
for i in tqdm_notebook(range(5,104)):
    RF = RandomForestRegressor(n_estimators=grid.best_params_['n_estimators'], max_features=grid.best_params_['max_features'],min_samples_leaf = grid.best_params_['min_samples_leaf'],max_depth = grid.best_params_['max_depth'],oob_score = False,random_state=grid.best_params_['random_state'],n_jobs=14)#random_state=grid.best_params_['random_state']   #最佳随机状态 = 2023
    RF.fit(var['X_'+str(i)], var['Y_'+str(i)].ravel())
    joblib.dump(RF,'E:\\OneDrive - HKUST Connect\\Research\\nh3_ml\\model\\RF_V6_alldatatraining_100bootstrap_1000Nmax_'+ str(i) +'.pkl')
    sdf.append(pt.inverse_transform(np.array(RF.predict(x_test).reshape(-1, 1)))[0])
print(np.mean(sdf))
print(np.std(sdf))
print(np.std(sdf)/np.mean(sdf))
print(np.percentile(sdf, 2.5 ))
print(np.percentile(sdf, 97.5 ))

<ipython-input-402-1c307f09e766>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(5,104)):


  0%|          | 0/99 [00:00<?, ?it/s]

6.389741306568287
0.733645165741682
0.11481609826481347
5.462807121008626
7.877703039367899


In [403]:
RF = RandomForestRegressor(n_estimators=grid.best_params_['n_estimators'], max_features=grid.best_params_['max_features'],min_samples_leaf = grid.best_params_['min_samples_leaf'],max_depth = grid.best_params_['max_depth'],oob_score = True,random_state=grid.best_params_['random_state'],n_jobs=14)

RandomForestRegressor(max_depth=20, max_features=11, n_estimators=500,
                      n_jobs=14, oob_score=True, random_state=2023)

In [405]:
from sklearn.model_selection import cross_val_score

In [406]:
scores = cross_val_score(RF, X, Y, cv=10)

E:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
E:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
E:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
E:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please ch

In [407]:
scores

array([0.75786924, 0.73029795, 0.70031377, 0.78173028, 0.67750061,
       0.75221561, 0.6973141 , 0.68132409, 0.67273229, 0.67676306])

In [408]:
scores.mean()

0.712806100756368

In [ ]:
RF.fit(X, Y.ravel())

In [410]:
##save model
joblib.dump(RF,'E:\\OneDrive - HKUST Connect\\Research\\nh3_ml\\model\\RF_V6_alldatatraining_100bootstrap_1000Nmax.pkl')
joblib.dump(grid,'E:\\OneDrive - HKUST Connect\\Research\\nh3_ml\\model\\RF_grid_V6_alldatatraining_100bootstrap_1000Nmax.pkl')
joblib.dump(pt,'E:\\OneDrive - HKUST Connect\\Research\\nh3_ml\\model\\boxcox_V6_alldatatraining_100bootstrap_1000Nmax.pkl')

['E:\\OneDrive - HKUST Connect\\Research\\nh3_ml\\model\\boxcox_V6_alldatatraining_100bootstrap_1000Nmax.pkl']